In [0]:
%run "../utils/mount_configuration"

In [0]:
tickets_price_df = spark.read.format("delta") \
                            .load(f"{processed_folder_path}/tickers_price")

In [0]:
display(tickets_price_df)

In [0]:
display(tickets_price_df.printSchema())

In [0]:
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go

# Convert PySpark DataFrame to Pandas
df_pandas = tickets_price_df.toPandas()

# Convert 'date' column to datetime format
df_pandas['date'] = pd.to_datetime(df_pandas['date'])

# Filter for a specific ticker
ticker_symbol = "GOOG"
df_filtered = df_pandas[(df_pandas["ticker"] == ticker_symbol) & (df_pandas["date"] >= datetime.today().replace(year=datetime.today().year - 1))]

# Sort by date
df_filtered = df_filtered.sort_values("date")

# Create candlestick chart with correct colors
fig = go.Figure(data=[go.Candlestick(
    x=df_filtered["date"],
    open=df_filtered["open_price"],
    high=df_filtered["highest_price"],
    low=df_filtered["lowest_price"],
    close=df_filtered["close_price"],
    increasing_line_color='red', 
    decreasing_line_color='green'  
)])

# Configure the chart
fig.update_layout(
    title=f'Candlestick Chart for {ticker_symbol}',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

# Display the chart
fig.show()